In [ ]:
# Import Libraries
import pandas as pd
from shapely.geometry import Point, MultiLineString, LineString
import xml.etree.ElementTree as ET
import pyproj
import re
import os
import geopandas as gpd
from shapely.ops import unary_union, nearest_points, substring
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import os
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiLineString, LineString
import xml.etree.ElementTree as ET
import pyproj

In [ ]:
# Function to load data, convert format (XML to Geopandas Dataframe; CSV to Geopandas Dataframe), Geospatial reference alignment, snapping process and divide the resulting dataframe into segments (e.g., the bus stop-to-stop segemnt here)

def process_bus_route_mass_conversion(input_directory='Route_Geometry', output_directory='Route_Geometry_GeoJSON', stops_directory='stops', stops_output_directory='Stops_GeoJSON'):
    # Task 1: Read xml and handle it in geojson with corresponding conversion
    # Loop through all date directories in the input directory
    for date_folder in os.listdir(input_directory):
        date_path = os.path.join(input_directory, date_folder)
        
        # Check if the date_path is a directory
        if not os.path.isdir(date_path):
            continue
        
        # Extract date from the folder name
        date_str = date_folder.split('_')[-1]

        # Loop through each route XML file within the date directory
        for file_name in os.listdir(date_path):
            if file_name.endswith('.xml'):
                route_path = os.path.join(date_path, file_name)

                # Extract route number and date from file name
                route_str = file_name.split('_')[2]  # assuming file name format is Route_Geometry_{route}_{date}.xml

                # Load and clean XML content
                with open(route_path, 'r', encoding='utf-8') as file:
                    xml_content = file.read()
                xml_content_no_ns = re.sub(r'xmlns(:\w+)?="[^"]+"', '', xml_content)
                xml_content_no_ns = re.sub(r'\b\w+:', '', xml_content_no_ns)
                root = ET.fromstring(xml_content_no_ns)

                # Extract data from XML and add to list
                data = []
                for route in root.findall('.//Route_Geometry'):
                    row = {
                        'Contract_Line_No': route.attrib.get('aContract_Line_No'),
                        'LBSL_Run_No': route.attrib.get('aLBSL_Run_No'),
                        'Sequence_No': route.attrib.get('aSequence_No'),
                        'Direction': route.find('Direction').text if route.find('Direction') is not None else None,
                        'Location_Easting': route.find('Location_Easting').text if route.find('Location_Easting') is not None else None,
                        'Location_Northing': route.find('Location_Northing').text if route.find('Location_Northing') is not None else None,
                        'Location_Longitude': route.find('Location_Longitude').text if route.find('Location_Longitude') is not None else None,
                        'Location_Latitude': route.find('Location_Latitude').text if route.find('Location_Latitude') is not None else None,
                        'Date': date_str,  # Add date column
                        'Route': route_str  # Add route column
                    }
                    data.append(row)

                # Create DataFrame and check for missing columns
                df = pd.DataFrame(data)
                expected_columns = ['Location_Easting', 'Location_Northing', 'Location_Longitude', 'Location_Latitude']
                for col in expected_columns:
                    if col not in df.columns:
                        df[col] = None  # Add missing columns as None if they don’t exist

                # Convert numeric columns
                df[expected_columns] = df[expected_columns].apply(pd.to_numeric, errors='coerce')

                # Convert df to GeoDataFrame
                df['geometry'] = df.apply(lambda row: Point(row['Location_Longitude'], row['Location_Latitude']) if pd.notnull(row['Location_Longitude']) and pd.notnull(row['Location_Latitude']) else None, axis=1)
                gdf = gpd.GeoDataFrame(df, geometry='geometry')
                gdf.set_crs(epsg=4326, inplace=True)

                # Remove rows with missing geometry
                gdf = gdf[gdf['geometry'].notna()]
                
                print(file_name)
                
                # Ensure 'Direction' column exists before grouping
                if 'Direction' not in gdf.columns:
                    print(f"No 'Direction' column found in {file_name}. Skipping this file...")
                    continue
                    
                # Filter rows where Direction is the same as LBSL_Run_No
                gdf = gdf[gdf['Direction'] == gdf['LBSL_Run_No']]

                # Group by Direction and create MultiLineString for each group
                multilines = []
                for direction, group in gdf.groupby('Direction'):
                    
                    # Extract coordinates as a list of tuples (longitude, latitude)
                    coords_direction = list(group.apply(lambda row: (row['Location_Longitude'], row['Location_Latitude']), axis=1))

                    # Create a LineString geometry from the coordinates
                    line_direction = LineString(coords_direction)

                    # Append to the multilines list
                    multilines.append({
                        'Direction': direction,
                        'geometry': line_direction
                    })
                                    
                # Create GeoDataFrame for MultiLineString geometries
                multilines_gdf = gpd.GeoDataFrame(multilines, geometry='geometry', crs="EPSG:4326")
                #multilines_gdf.set_crs(epsg=4326, inplace=True)
                multilines_gdf['Direction'] = multilines_gdf['Direction'].astype(int)
                
                #print(multilines_gdf)
                
                # Task 2: Process stops file
                stops_file_path = os.path.join(stops_directory, f'Data_3rdParties_bus-sequences-{date_str}.csv')
                if os.path.exists(stops_file_path):
                    stops_df = pd.read_csv(stops_file_path)

                    # Filter stops DataFrame by Route
                    stops_df = stops_df[stops_df['Route'] == route_str]
                    
                    # Skip this route if no stops data is available for it
                    if stops_df.empty:
                        print(f"No stops data found for route {route_str}. Skipping...")
                        continue
                    
                    # Define the input (OSGB36 / British National Grid) and output coordinate reference system (WGS84)
                    transformer = pyproj.Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)
                    # Step 1: Apply the transformer to convert easting and northing to longitude and latitude
                    stops_df[['Longitude', 'Latitude']] = stops_df.apply(lambda row: pd.Series(transformer.transform(row['Location_Easting'], row['Location_Northing'])), axis=1)
                
                    # Step 2: Create a new column called 'geometry' using the longitude and latitude columns
                    stops_df['geometry'] = stops_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

                    # Step 3: Convert the DataFrame to a GeoDataFrame
                    stops_gdf = gpd.GeoDataFrame(stops_df, geometry='geometry', crs="EPSG:4326")
                    stops_gdf = stops_gdf.reset_index()
                    stops_gdf = stops_gdf.drop(columns = ['index'])
                    
                    # Step 4: Add info of required columns
                    # Create columns for 'from_stop_Name' and 'to_stop_Name'
                    stops_gdf['from_stop_Name'] = stops_gdf['Stop_Name']
                    stops_gdf['to_stop_Name'] = stops_gdf['Stop_Name'].shift(-1)
                    # Create columns for 'from_stop_code' and 'to_stop_code'
                    stops_gdf['from_stop_code'] = stops_gdf['Stop_Code_LBSL']
                    stops_gdf['to_stop_code'] = stops_gdf['Stop_Code_LBSL'].shift(-1)
                    # Create columns for 'sequence_from_stop' and 'sequence_to_stop'
                    stops_gdf['sequence_from_stop'] = stops_gdf['Sequence']
                    stops_gdf['sequence_to_stop'] = stops_gdf['Sequence'].shift(-1)
                    stops_gdf.sequence_to_stop = stops_gdf.sequence_to_stop.fillna(-99)
                    stops_gdf['sequence_to_stop'] = stops_gdf['sequence_to_stop'].astype(int)
                    stops_gdf['Run'] = stops_gdf['Run'].astype(int)
                    
                # Task 3: Snap stops to corresponding route geometry
                for direction in multilines_gdf['Direction'].unique():
                    direction = int(direction)
                    route_gdf_direction = multilines_gdf[multilines_gdf['Direction'] == direction]
                    stops_gdf_direction = stops_gdf[stops_gdf['Run'] == direction]

                    # Skip if no stops data exists for this direction
                    if route_gdf_direction.empty or stops_gdf_direction.empty:
                        continue

                    # Combine all geometries in 'geometry' column into a single MultiLineString
                    route = unary_union(route_gdf_direction['geometry'].tolist())                        
                        
                    # Check if route was successfully created
                    if route.is_empty:
                        print(f"Route geometry for route {route_str} direction {direction} is empty. Skipping...")
                        continue
                        
                    # Convert MultiLineString to LineString if necessary
                    if isinstance(route, MultiLineString):
                        # Merge the MultiLineString into a single LineString
                        route = LineString([point for line in route for point in line.coords])

                    # Snap each stop to the nearest point on the route
                    def snap_to_route(stop):
                        # Check if stop is a valid geometry
                        if stop is None or stop.is_empty:
                            return None
                        # Calculate nearest point only if stop and route are valid
                        nearest_point = nearest_points(route, stop)[0]
                        return nearest_point

                    # Apply snapping to get the nearest point on the route for each stop
                    stops_gdf_direction['snapped'] = stops_gdf_direction.geometry.apply(snap_to_route)

                    # Calculate the distance along the route for each stop
                    def distance_along_route(stop):
                        if stop is None or stop.is_empty:
                            return None
                        return route.project(stop)
                    
                    # Distance from the first point on the route
                    stops_gdf_direction['stop_distance_from_1st'] = stops_gdf_direction['snapped'].apply(distance_along_route)

                    # Function to create route geometry between consecutive stops
                    def get_segment_geometry(row):
                        next_row_index = row.name + 1
                        if next_row_index not in stops_gdf_direction.index:  # Last row check
                            return None

                        # Start and end distances along the route for the segment
                        start_distance = row['stop_distance_from_1st']
                        end_distance = stops_gdf_direction.loc[next_row_index, 'stop_distance_from_1st']

                        # Check and log invalid distances for debugging
                        if start_distance is None or end_distance is None:
                            print(f"Missing distance for row {row.name}")
                            return None

                        # Ensure valid order for distances
                        if start_distance >= end_distance:
                            print(f"Reversed or identical distances at row {row.name}: start_distance={start_distance}, end_distance={end_distance}")
                            return None

                        # If route is a MultiLineString, apply substring separately for each LineString part
                        if isinstance(route, MultiLineString):
                            segments = [substring(line, start_distance, end_distance) for line in route.geoms if line.length >= end_distance - start_distance]
                            segment = segments[0] if segments else None
                        else:
                            # Use substring directly if route is already a LineString
                            segment = substring(route, start_distance, end_distance)

                        return segment

                    # Drop snapped column
                    stops_gdf_direction = stops_gdf_direction.drop(columns = ['snapped'])
                    
                    # Apply the function to each row to create precise segment geometries
                    stops_gdf_direction['geometry'] = stops_gdf_direction.apply(get_segment_geometry, axis=1)
                    
                    # Drop unnecessary column
                    stops_gdf_direction = stops_gdf_direction.drop(columns = ['stop_distance_from_1st'])
                    
                    # Create output path and save as GeoJSON
                    output_path = os.path.join(output_directory, date_folder, route_str, str(direction))
                    os.makedirs(output_path, exist_ok=True)
                    output_file = os.path.join(output_path, f'{file_name.replace(".xml", ".geojson")}')
                    stops_gdf_direction.to_file(output_file, driver='GeoJSON')
                    print(f'Saved GeoJSON: {output_file}')
                
                
# Run the function
process_bus_route_mass_conversion()


In [ ]:
# Checking of output (No point geometry should be returned)

def check_geojson_files(directory):
    files_checked = 0
    point_files = []

    # Confirm that the directory exists
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return

    print(f"Scanning directory: {directory}")

    # Traverse through the given directory
    for root, dirs, files in os.walk(directory):
        print(f"Entering directory: {root}")  # Debug: Show current directory
        for file in files:
            if file.lower().endswith('.geojson'):
                file_path = os.path.join(root, file)
                print(f"Checking file: {file_path}")  # Debug: Show file being checked
                
                try:
                    # Load the GeoJSON file using geopandas
                    gdf = gpd.read_file(file_path)
                    
                    # Check if the geometry type is Point
                    if gdf.geometry.geom_type.eq('Point').any():
                        point_files.append(file_path)
                
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
                
                files_checked += 1

    # Reporting results
    if files_checked == 0:
        print("\nNo GeoJSON files found in the directory.")
    elif point_files:
        print("\nFiles with Point geometry found:")
        for point_file in point_files:
            print(point_file)
    else:
        print("\nNo files with Point geometry found.")
    
    print(f"\nTotal files checked: {files_checked}")

# Directory path
directory = "Route_Geometry_GeoJSON/Route_Geometry_20231208"
check_geojson_files(directory)
